In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time
from IPython.display import clear_output
import os

import scipy
import matplotlib

from robot import *
from utils import *
import pybullet as p
import pybullet_data
import networkx as nx
from functools import partial

%load_ext autoreload
%autoreload 2

## Setup Pybullet

In [2]:
# name_in =  'urdf/support.obj'
# name_out = 'urdf/support_concave.obj'
# name_in =  'urdf/support_cylindrical_col.obj'
# name_out = 'urdf/support_cylindrical_colconc.obj'
# name_log = "log.txt"
# p.vhacd(name_in, name_out, name_log, alpha=0.04,resolution=1000000 )

Now 'urdf/support_cylindrical.urdf' shows the structure real visualization but with the collision shape constructed from the command p.vhacd(.)

In [3]:
physics_client_id = p.connect(p.GUI)
p.setPhysicsEngineParameter(enableFileCaching=0)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)


In [4]:
p.resetSimulation()

### Setup Robot

In [5]:
robot_urdf = 'urdf/panda/panda_arm_rivet.urdf';
robot_id = p.loadURDF(fileName=robot_urdf)
dof = p.getNumJoints(robot_id)

plane_id = p.loadURDF('plane.urdf')
pb_joint_indices = np.arange(7)
joint_limits = get_joint_limits(robot_id, pb_joint_indices)


set_q_std = partial(set_q,robot_id, pb_joint_indices)
vis_traj_std = partial(vis_traj, vis_func= set_q_std)

### Visualize the big frame

In [6]:
is_simple = False
obs_ids = []
rgbacolor = (0,0,1,0)

board_id = p.loadURDF('urdf/support_cylindrical.urdf', globalScaling=0.0008, useFixedBase=1)
quat = w2quat(np.array([0,0,-np.pi/2]))
p.resetBasePositionAndOrientation(board_id, (0.6,-2.2,-.3), quat)

offset = 0.

In [7]:
x_board = 0.47
p.resetBasePositionAndOrientation(board_id, (x_board,-1.7,-.41), quat)

### Setup targets in the middle

In [8]:
ys = np.concatenate([np.arange(-0.34, -0.15, 0.05), np.arange(0.2, 0.36, 0.05)],-1)
zs = np.arange(0.35, 0.54, 0.05) + 0.2
x = x_board + 0.3
targets_mid = []
target_ids = []
offset = -0.05
for y in ys:
    for z in zs:
        target = np.array([x, y,z])
        targets_mid += [target]
        _,_,target_id = create_primitives(p.GEOM_BOX, halfExtents=(0.03, 0.01, 0.01), rgbaColor=(1,0,0,1), baseMass=0)
        target_ids += [target_id]
        p.resetBasePositionAndOrientation(target_id, targets_mid[-1], (0,0,0,1))
        targets_mid[-1][0] -= 0.03

targets_mid = np.array(targets_mid)

### Targets on the bottom

In [9]:
R = w2mat(quat2w(euler2quat([0,10,0.])))

zs = np.arange(0.25,0.31, 0.05) + 0.2
#ys = np.array([-0.45, -0.25,  0.15,  0.35])
#zs = np.arange(0.6, 0.81, 0.1)
x = x_board + 0.25
targets_down = []
target_ids = []
offset = 0.03
for y in ys:
    offset_x = 0.
    for z in zs:
        target = np.array([x+offset_x, y,z])
        targets_down += [target]
        _,_,target_id = create_primitives(p.GEOM_BOX, halfExtents=(0.03, 0.01, 0.01), rgbaColor=(0,0,1,1), baseMass=0)
        target_ids += [target_id]
        p.resetBasePositionAndOrientation(target_id, targets_down[-1], (0,0,0,1))
        targets_down[-1] += (R@np.array([1,0,0]))*offset
        offset_x += 0.03

targets_down = np.array(targets_down)

for target_id in target_ids:
    p.resetBasePositionAndOrientation(target_id,posObj=p.getBasePositionAndOrientation(target_id)[0 ], ornObj=euler2quat([0,10,0.]).tolist())

### Targets on the top

In [10]:
R = w2mat(quat2w(euler2quat([0,-10,0.])))

zs = np.arange(0.55,0.61, 0.05) + 0.2

x = x_board + 0.27
targets_up = []
target_ids = []
offset = 0.03
for y in ys:
    offset_x = 0.
    for z in zs:
        target = np.array([x+offset_x, y,z])
        targets_up += [target]
        _,_,target_id = create_primitives(p.GEOM_BOX, halfExtents=(0.03, 0.01, 0.01), rgbaColor=(0,1,0,1), baseMass=0)
        target_ids += [target_id]
        p.resetBasePositionAndOrientation(target_id, targets_up[-1], (0,0,0,1))
        targets_up[-1] += (R@np.array([1,0,0]))*offset
        offset_x -= 0.03

targets_up = np.array(targets_up)

for target_id in target_ids:
    p.resetBasePositionAndOrientation(target_id,posObj=p.getBasePositionAndOrientation(target_id)[0], ornObj=euler2quat([0,-10,0.]).tolist())

### RRT

In [11]:
rob_sampler = sampler(joint_limits)
rob_col_checker = col_checker(robot_id, pb_joint_indices, [ plane_id] + [board_id])
rob_interpolator = interpolator()

rrt = BiRRT(7, rob_sampler, rob_col_checker, rob_interpolator)

## Load Data

In [12]:
# Loading the rivet positions
targets_x = np.load("data/targets_x.npy", allow_pickle=True)
targets_orn = np.load("data/targets_orn.npy", allow_pickle=True)
rotation_axes = np.load('data/rotation_axes.npy', allow_pickle=True)
labels = targets_x.item().keys()
hole_nb = 0
for label in labels:
    hole_nb += targets_x.item()[label].shape[0]
print(hole_nb)
targets = np.concatenate([targets_x.item()[name] for name in labels],0)
# Loading the joint configurations
data = np.load('data/riveting_results70.npy', allow_pickle=True)[()]
qs_raw = data['qs']
disp = np.linalg.norm(data['disp'], axis=2)
disp = data['disp'][:,:,0]


data_before = np.load('data/before_riveting_results70.npy', allow_pickle=True)[()]
qs_raw_before = data_before['qs']
disp_before = np.linalg.norm(data_before['disp'], axis=2)
disp_before = data_before['disp'][:,:,0]

64


#### Filter the data (remove the one with collision)

In [13]:
qs = []
disps = []
qs_lens = []
for i,qset in enumerate(qs_raw):
    qlist = []
    displist = []
    for j,q in enumerate(qset):
        if rob_col_checker.check_collision(q) is False:
            qlist += [q]
            displist += [abs(disp[i][j])]
    if len(qlist) <= 50: 
        print(i, len(qlist))
        qset = qs_raw_before[i]
        for j,q in enumerate(qset):
            if rob_col_checker.check_collision(q) is False:
                qlist += [q]
                displist += [abs(disp_before[i][j])]
            
    qlist = np.array(qlist)
    displist = np.array(displist)
        
    qs += [qlist]
    disps += [displist]
    qs_lens += [len(qlist)]
#clear_output()

0 20
8 50
10 42
12 45
14 21
19 47
51 35
53 38
54 50
55 39
56 9
57 21
58 34
59 21
60 24
62 37


#### Visualize the data

In [14]:
class time_struct():
    def __init__(self):
        self.select_config = 0
        self.plan = 0
        self.postplan = 0

def plan_motion(q_init, path, qs, disps, max_config, w_disp = 20, max_try = 20):
    comp_time = time_struct()
    tic = time.time()
    print('Obtaining configurations in the required order')
    qs_new = [q_init]
    disps_new = [np.array([0])]
    for idx in path:
        qs_new += [qs[idx][:max_config]]
        disps_new += [disps[idx][:max_config]]
    qs_new += [q_init]
    disps_new += [np.array([0])]
    
    print("Construct cgraph")
    cgraph = construct_cgraph(qs_new, disps_new, w_disp = w_disp)
    
    print('Plan configuration path')
    cpath = nx.dijkstra_path(cgraph, 0, len(cgraph.nodes()) - 1)
    traj, total_disp = extract_traj(cgraph, cpath)
    clear_output()
    toc = time.time()
    
    comp_time.select_config = toc-tic
    comp_time.plan = 0
    comp_time.postplan = 0
    
    print('Plan motion')
    trajs = []
    total_dist = 0
    for i in range(0,len(cpath)-2):
        q = cgraph.nodes[cpath[i]]['value']
        qn = cgraph.nodes[cpath[i+1]]['value']
        tic = time.time()
        traj,_,_ = rrt.plan(q,qn)
        toc = time.time()
        comp_time.plan += toc-tic
        shortpath = rrt.shortcut_path(traj, max_try = max_try)
        shorttraj = np.array(traj)[shortpath]
        shortpath2 = rrt.shortcut_path_old(shorttraj,step_length=0.05)
        shorttraj2 = rrt.interpolate_traj(shortpath2, shorttraj)
        toc2 = time.time()
        comp_time.postplan += toc2-toc
        trajs += [shorttraj2]
        total_dist += calc_dist(shorttraj2)
        clear_output()
        print(i)

    clear_output()
    print('Planning time:{}, {}, {}'.format(comp_time.select_config, comp_time.plan, comp_time.postplan))
    return trajs, total_dist, total_disp, comp_time

In [15]:
def plan_motion_max(q_init, path, qs, disps, max_config, w_disp = 0.3, max_try = 20):
    comp_time = time_struct()
    tic = time.time()
    print('Obtaining configurations that minimizes d') 
    qs_new = [q_init[0]]
    disps_new = [np.array([0])]
    total_disp = 0
    for idx in path:
        disp = disps[idx]
        idx_min = np.argmin(disp[:max_config])
        total_disp += disp[idx_min]
        qs_new += [qs[idx][idx_min]]
        disps_new += [disps[idx][idx_min]]
    disps_new += [np.array([0])]
    qs_new += [q_init[0]]
    toc = time.time()

    comp_time.select_config = toc-tic
    comp_time.plan = 0
    comp_time.postplan = 0

    
    clear_output()
    print('Plan the motion')
    trajs = []
    total_dist = 0
    for i in range(len(path)):
        q = qs_new[i]
        qn = qs_new[i+1]
        tic = time.time()
        traj,_,_ = rrt.plan(q,qn)
        toc = time.time()
        comp_time.plan += toc-tic
        
        shortpath = rrt.shortcut_path(traj, max_try = max_try)
        shorttraj = np.array(traj)[shortpath]
        shortpath2 = rrt.shortcut_path_old(shorttraj,step_length=0.05)
        shorttraj2 = rrt.interpolate_traj(shortpath2, shorttraj)
        toc2 = time.time()
        comp_time.postplan += toc2-toc
        trajs += [shorttraj2]
        total_dist += calc_dist(shorttraj2)
        clear_output()
        print(i)


    clear_output()
    print('Planning time:{}, {}, {}'.format(comp_time.select_config, comp_time.plan, comp_time.postplan))
    return trajs, total_dist, total_disp, comp_time

### Construct Graph & Solve TSP

In [16]:
from rtsp import *
#graph = from_coordinate_list(targets, euclidean_metric)
#path = two_opt(graph) #optimize the hole ordering, if necessary

path = np.array([2, 4, 6, 8, 1, 3, 5, 7, 
                 20, 24, 28, 32, 19, 23, 27, 31, 
                 18, 22, 26, 30, 17, 21, 25, 29, 
                 50, 52, 54, 56, 49, 51, 53, 55, 
                 10, 12, 14, 16, 9, 11, 13, 15, 
                 36, 40, 44, 48, 35, 39, 43, 47,
                 34, 38, 42, 46, 33, 37, 41, 45, 
                 58, 60, 62, 64, 57, 59, 61, 63])-1 #we use prespecified hole ordering in this work

#### Set initial configuration

In [17]:
max_config = 100
# q_init = np.array([ 1.23170025, -1.10558688,  0.22936525, -2.28523573, -1.07928728,
#         2.17303381, -1.24231352])[None,:]
#q_init = qs[0][7][None,:]
qs_outside = np.load('data/qs_outside.npy', allow_pickle = True)
q_init = qs_outside[4][None,:]
set_q_std(q_init[0])

#### Determine the weight (by evaluating several possible weights and choose the best one)

In [ ]:
ws = [0.1, 1, 10, 30]
rtsp_fixed_dists = dict()
rtsp_fixed_disps = dict()
rtsp_fixed_trajs = dict()
rtsp_fixed_time = dict()

for n in ws:
    rtsp_fixed_trajs[n], rtsp_fixed_dists[n], rtsp_fixed_disps[n], rtsp_fixed_time[n] = plan_motion(q_init, path, qs, disps, 100, w_disp = n)

plt.plot(list(rtsp_fixed_dists.keys()), list(rtsp_fixed_dists.values()), '*')

plt.plot(list(rtsp_fixed_disps.keys()), list(rtsp_fixed_disps.values()), '*')

data = dict()
data['rtsp_fixed'] = [rtsp_fixed_dists, rtsp_fixed_disps, rtsp_fixed_trajs]
data['ws'] = ws
np.save('data/rivet_planning_cylindrical_computeweight.npy', data, allow_pickle=True)

In [19]:
num_configs = [3, 5, 20, 50, 100, 200]

#### Planning with RTSP

In [20]:
rtsp_fixed_dists = dict()
rtsp_fixed_disps = dict()
rtsp_fixed_trajs = dict()
rtsp_fixed_time = dict()
rtsp_fixed_config_time = []
rtsp_fixed_plan_time = []
rtsp_fixed_postplan_time = []
w_disp = 10 #selected from above

In [ ]:
for n in num_configs:
    print(n)
    print('Planning with {} configs'.format(n))
    rtsp_fixed_trajs[n], rtsp_fixed_dists[n], rtsp_fixed_disps[n], rtsp_fixed_time[n] = plan_motion(q_init, path, qs, disps, n, w_disp = w_disp)
    rtsp_fixed_config_time += [rtsp_fixed_time[n].select_config]
    rtsp_fixed_plan_time += [rtsp_fixed_time[n].plan]
    rtsp_fixed_postplan_time += [rtsp_fixed_time[n].postplan]

In [ ]:
for traj in rtsp_fixed_trajs[3]:
    vis_traj_std(traj,dt=0.02)
    time.sleep(0.1)

In [121]:
data = dict()
data['rtsp_fixed'] = [rtsp_fixed_dists, rtsp_fixed_disps, rtsp_fixed_trajs, rtsp_fixed_config_time,rtsp_fixed_plan_time, rtsp_fixed_postplan_time ]
data['num_configs'] = num_configs
np.save('data/rivet_planning_cylindrical_rtsp2.npy', data, allow_pickle=True)

#### Planning with baseline

In [122]:
baseline_fixed_dists = dict()
baseline_fixed_disps = dict()
baseline_fixed_trajs = dict()
baseline_fixed_time = dict()
baseline_fixed_config_time = []
baseline_fixed_plan_time = []
baseline_fixed_postplan_time = []
w_disp = 10

In [136]:
for n in num_configs:
    print(n)
    print('Planning with {} configs'.format(n))
    baseline_fixed_trajs[n], baseline_fixed_dists[n], baseline_fixed_disps[n], baseline_fixed_time[n] = plan_motion_max(q_init, path, qs, disps, n, w_disp = w_disp)
    baseline_fixed_config_time += [baseline_fixed_time[n].select_config]
    baseline_fixed_plan_time += [baseline_fixed_time[n].plan]
    baseline_fixed_postplan_time += [baseline_fixed_time[n].postplan]

Planning time:0.0008473396301269531, 303.05488562583923, 522.3013753890991


In [ ]:
for traj in baseline_fixed_trajs[200]:
    vis_traj_std(traj,dt=0.02)
    time.sleep(0.1)

In [138]:
data = dict()
data['rtsp_fixed'] = [baseline_fixed_dists, baseline_fixed_disps, baseline_fixed_trajs, baseline_fixed_config_time,baseline_fixed_plan_time, baseline_fixed_postplan_time ]
#data['num_configs'] = num_configs
np.save('data/rivet_planning_cylindrical_baseline3.npy', data, allow_pickle=True)